In [ ]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
import importlib as imp

imp.reload(lt)

In [ ]:
device = "cpu"
dtype = torch.float32

resolution=100
lattice = lt.Lattice(lt.D2Q9, device, dtype)
#flow = lt.PoiseuilleFlow2D(resolution=resolution, reynolds_number=10000, mach_number=0.029, lattice=lattice, initialize_with_zeros=False)
#flow = lt.Vortex2D(resolution=100, reynolds_number=10000, mach_number=0.1, lattice=lattice, vortex_radius = 0.05, vortex_strength = 0.4)
flow = lt.DoublyPeriodicShear2D(resolution=200, reynolds_number=5000, mach_number=0.1, lattice=lattice)

#grid = [np.linspace(0, 2 * np.pi, num=flow.resolution, endpoint=False) for _ in range(lattice.D)]


# def grid():
#     x = np.linspace(0, 1, num=resolution + 1, endpoint=True)
#     y = np.linspace(0, 1, num=resolution + 1, endpoint=True)
#     return np.meshgrid(x, y, indexing='ij')



#p, u = flow.initial_solution(grid())

# def grid():
#     x = np.linspace(0, 1, num=resolution + 1, endpoint=True)
#     y = np.linspace(0, 1, num=resolution + 1, endpoint=True)
#     return np.meshgrid(x, y, indexing='ij')

# x,y = grid()

# half_lattice_spacing = 0.5 / resolution

# nu = 0.000015
# rho = 1
# u = np.array([
#             0.001 / (2 * 1 * nu) * ((y - half_lattice_spacing) * (1 - half_lattice_spacing - y)),
#             np.zeros(x.shape)
#         ])
# p = np.array([y * 0 + rho])

# np.shape(y)

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1,2)
# im1 = ax1.imshow(u[0])
# fig.colorbar(im1, ax=ax1, shrink=0.5)
# im2 = ax2.imshow(p[0])
# fig.colorbar(im2, ax=ax2, shrink=0.5)

# #plt.title('Initialized velocity')
# plt.tight_layout()
# plt.show()

In [ ]:
num_steps = 5000
num_save = 10
interval = np.ceil(num_steps/num_save)

In [ ]:
transform = lt.D2Q9Cascaded(lattice)
tau = flow.units.relaxation_parameter_lu
relaxation_parameters = np.array([1, 1, 1, tau, tau, tau, 1, 1, 1])
#collision = lt.BGKCollision(lattice, tau=flow.units.relaxation_parameter_lu)
collision = lt.CMCollision(lattice, transform, relaxation_parameters)
streaming = lt.StandardStreaming(lattice)
simulation = lt.Simulation(flow=flow, lattice=lattice, collision=collision, streaming=streaming)

simulation.reporters.append(lt.VTKReporter(lattice, flow, interval, filename_base="output/output"))
# u  = flow.units.convert_velocity_to_pu(lattice.u(simulation.f)).numpy()
# u = u[0] # x-velocity
# np.max(u)

In [ ]:
# rho  = flow.units.convert_density_to_pu(lattice.rho(simulation.f)).numpy()
# rho = rho[0]-1

# np.shape(rho)

In [ ]:
# y = np.linspace(0, 1, num=101, endpoint=True)
# y = y - 0.5/100
# print(y)

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1,2)
# im1 = ax1.imshow(u[0])
# fig.colorbar(im1, ax=ax1, shrink=0.5)
# im2 = ax2.imshow(p)
# fig.colorbar(im2, ax=ax2, shrink=0.5)

# #plt.title('Initialized velocity')
# plt.tight_layout()
# plt.show()

In [ ]:
simulation.initialize_f_neq()
mlups = simulation.step(num_steps)
print("Performance in MLUPS:", mlups)

In [ ]:
u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f)).numpy()
u_norm = np.linalg.norm(u,axis=0)
plt.imshow(u_norm)
plt.colorbar()
plt.show()